# Updating YAML Files with OpenAI

Below is a (javascript) Jupyter notebook which goes through the flow 
of processing [this GitHub issue](https://github.com/robotsail/apps-clone/issues/1) 
on a cloned version of [operate-first/apps](https://github.com/robotsail/apps-clone).

## Table of Contents

The flow of the jupyter notebook is as follows:

1. Setup
2. Querying the issue from GitHub
3. Processing its contents & downloading the relevant files
4. Generating a prompt for OpenAI
5. Mapping the response into the updated files

## Setup

Below we define two objects:

- `v`: A JavaScript object to store the variables for this notebook
- `f`: A JavaScript object to store the functions for this notebook


In [1]:
// object for variables so we don't run into problems when redclaring
const v = {
	// store for variables we obtained in asynchronous calls
	asynced: {},
};
// functions
const f = {};

undefined

### Constants
We define a table of constants within `v`, where we'll place our API keys

In [2]:
v.const = {
	REPO_OWNER: 'RobotSail',
	REPO_NAME: 'apps-clone',
	OPENAI_API_KEY: process.env.OPENAI_API_KEY,
	OPENAI_API_URL: 'https://api.openai.com/v1/engines/code-davinci-001/completions',
	OPENAI_STOP_SEQUENCES: ['####'],
	GITHUB_API_KEY: process.env.GITHUB_PAT,
}
// hide the output 
console.log('output hidden')

output hidden


undefined

### GraphQL

To cut down on the amount of requests made to GitHub and avoid rate-limiting issues,
we'll use the GraphQL API to query the necessary data from GitHub.
Below, we're using the `urql` GraphQL client, but we can equally use Apollo or just straight request with Axios.

In [3]:
var { gql } = require('@urql/core');
var { createClient } = require('@urql/core');
require('isomorphic-unfetch');

v.client = createClient({
    url: 'https://api.github.com/graphql',
    fetchOptions: () => {
        const gh_token = v.const.GITHUB_API_KEY;
        return {
            headers: { Authorization: gh_token ? `Bearer ${gh_token}` : '' }
        };
    },
});

Client {
  url: 'https://api.github.com/graphql',
  fetchOptions: [Function: fetchOptions],
  fetch: undefined,
  suspense: false,
  requestPolicy: 'cache-first',
  preferGetMethod: false,
  maskTypename: false,
  'operations$': [Function: source],
  reexecuteOperation: [Function: reexecuteOperation],
  createOperationContext: [Function: createOperationContext],
  createRequestOperation: [Function: createRequestOperation],
  executeRequestOperation: [Function: executeRequestOperation],
  executeQuery: [Function: executeQuery],
  executeSubscription: [Function: executeSubscription],
  executeMutation: [Function: executeMutation],
  query: [Function: query],
  readQuery: [Function: readQuery],
  subscription: [Function: subscription],
  mutation: [Function: mutation],
  subscribeToDebugTarget: [Function (anonymous)]
}

## Obtaining Issues

We'll create a GraphQL query to obtain the issues we need. This query is simple,
but can be easily scaled when implementing this as a bot.

In [4]:
v.issuesQuery = gql`
# grab first 100 issues in operate first repo
query GetIssues ($owner: String!, $repo: String!, $limit: Int, $labels: [String!]) {
  repository(owner: $owner, name: $repo) {
    issues(first: $limit, labels: $labels) {
      totalCount
      pageInfo {	
        startCursor
        hasNextPage
        endCursor
      }
      edges {
        node {
          number
          body
          title
          author {
            login
          }
        }
      }
    }
  }
}
`;

{
  kind: 'Document',
  definitions: [
    {
      kind: 'OperationDefinition',
      operation: 'query',
      name: [Object],
      variableDefinitions: [Array],
      directives: [],
      selectionSet: [Object]
    }
  ],
  loc: {
    start: 0,
    end: 290,
    source: {
      body: '# GetIssues\n' +
        'query GetIssues($owner: String! $repo: String! $limit: Int $labels: [String!]) { repository(owner: $owner name: $repo) { issues(first: $limit labels: $labels) { totalCount pageInfo { startCursor hasNextPage endCursor } edges { node { number body title author { login } } } } } }',
      name: 'gql',
      locationOffset: [Object]
    }
  },
  __key: 3955687553
}

In [5]:
// get the first N issues
f.getIssues = async (gqlClient, limit = 1) => {
	// build query 
	const res = await gqlClient.query(v.issuesQuery, {
		owner: v.const.REPO_OWNER,
		repo: v.const.REPO_NAME,
		limit: limit
	})
		.toPromise()
		.then(r => {
			console.log(r);
			return r.data
		})
		.catch(e => console.log('error', e));
	await res
	v.asynced.issues = res.repository.issues.edges.map(({ node }) => {
		if (node.body !== undefined) {
			node.body = node.body.replace(/\r\n/g, '\n');
		}
		return {
			number: node.number,
			body: node.body,
			title: node.title,
		}
	});
	return v.asynced.issues;
};

// needed to run async functions
$$.async();

{
	f.getIssues(v.client, 1);	
	console.log('issues', v.asynced.issues);
	$$done$$();
}


issues undefined


## Obtaining File Data
Normally, this would all be handled inside of one function, but IJavaScript does not have the best support for async,
so we're breaking it up into multiple functions before passing the final data over to buildPrompt.

Once we have the issue, we simply extract the filepaths and generate a GraphQL query that'll obtain all of the files with a single request.

In [6]:
// return a map of {fileName => {path: path, content: content}}
f.getFilesFromIssue = (issue) => {
	// extract a map of the files from the issue based on the following regex:
	// /`(@([a-zA-Z0-9_\-]+):(.+))`/g
	const fileRegex = /`(@([a-zA-Z0-9_\-]+):(.+))`/g;
	// create a map from the filename to the filepath
	const fileMap = new Map();
	// extract the string from group 3 of the regex
	let match;
	while (match = fileRegex.exec(issue)) {
		let [name, path] = [match[2], match[3]];
		if (!fileMap.has(name)) {
			// define the file object here 
			fileMap.set(name, {
				path: path,
				content: '',
				updatedContent: '',
			});
		} else {
			console.error(`duplicate file name ${name}`);
		}
	}
	return fileMap;
}


[Function (anonymous)]

In [8]:
// return a list of files
v.filesFromIssue = f.getFilesFromIssue(v.asynced.issues[0].body);

Map(2) {
  'file1' => {
    path: 'grafana/base/datasource.yaml',
    content: '',
    updatedContent: ''
  },
  'file2' => {
    path: 'grafana/base/grafana-route.yaml',
    content: '',
    updatedContent: ''
  }
}

In [9]:
f.createFileQuery = (files, branch) => {
	let fileQuery = `
	fragment contents on Blob {
		text
	}
	
	fragment rateLimitInfo on Query {
		rateLimit {
			limit
			cost
			remaining
			resetAt
		}
	}
	
	query GetFiles($Owner: String!, $Repo: String!) {	
		...rateLimitInfo
		repository(owner: $Owner, name:$Repo) {
	`;
	
		const addFilesToQuery = (query, fileMap, branch) => {
			// iterate through fileMap and add each file to the query string
			fileMap.forEach((file, name) => {
				query += `
				${name}: object(expression: "${branch}:${file.path}") {
					...contents
				}
	`;
			});
			return query;
		}
	
		fileQuery = addFilesToQuery(fileQuery,files, branch);
		fileQuery += `
		}
	}
	`;
	return fileQuery;
}	


[Function (anonymous)]

In [10]:
f.downloadFilesFromBranch = async (gqlClient, files, branch) => {
	// first we must build the GraphQL query to get the files
	const fileQuery = f.createFileQuery(files, branch);
	const fileQueryGql = gql`${fileQuery}`;

	// then we can run the query
	const res = await gqlClient.query(fileQueryGql, {
		Owner: v.const.REPO_OWNER,
		Repo: v.const.REPO_NAME
	})
		.toPromise()
		.then(r => {
			if (r.data !== undefined && r.data.repository !== undefined) {
				for (const [fileName, file] of files) {
					file.content = r.data.repository[fileName].text;					
				}
			}
		})


	const promises = [res,];
	Promise.all(promises);
	return files;
}

[AsyncFunction (anonymous)]

In [11]:
$$.async();
{
	f.downloadFilesFromBranch(v.client, v.filesFromIssue, 'main');
}

Map(2) {
  'file1' => {
    path: 'grafana/base/datasource.yaml',
    content: 'apiVersion: integreatly.org/v1alpha1\n' +
      'kind: GrafanaDataSource\n' +
      'metadata:\n' +
      '  name: datasource\n' +
      'spec:\n' +
      '  name: prometheus-grafanadatasource.yaml\n' +
      '  datasources:\n' +
      '    - name: Prometheus\n' +
      '    - access: proxy\n' +
      '      editable: true\n' +
      '      isDefault: true\n' +
      '      jsonData:\n' +
      "        httpHeaderName1: 'Authorization'\n" +
      '        timeInterval: 5s\n' +
      '        tlsSkipVerify: true\n' +
      '      name: Prometheus\n' +
      '      secureJsonData:\n' +
      "        httpHeaderValue1: 'Bearer ${BEARER_TOKEN}'\n" +
      '      type: prometheus\n' +
      "      url: 'https://thanos-querier.openshift-monitoring.svc.cluster.local:9091'\n",
    updatedContent: ''
  },
  'file2' => {
    path: 'grafana/base/grafana-route.yaml',
    content: 'kind: Route\n' +
      'apiVersion: ro

## Building the Prompt

Now that we have the issue from GitHub, and have downloaded the necessary files, we can build the prompt.

The following is an example prompt that would be sent to OpenAI;
```
	==== BEGIN PROMPT ====
	# Listed below are:
	# 1. Explanation of how two files need to be changed
	# 2. The original files, separated by a '---' string
	# 3. An updated version of the files with the described changes, with each file separated by a '---' string
	# 4. A '####' string, indicating the end of this document


	## Description of issues:
	@file1:/path/to/file1
	@file2:/path/to/file2

	The amount of CPU in @file1 needs to be increased to 512M
	The PVC storage amount in @file2 should be decreased to 10Gi

	## Original files:
	# @file1
	<contents of file1>
	---
	# @file2
	<contents of file2>

	## Updated files:
	==== END PROMPT ====
```


In [12]:
f.buildPrompt = (issue, files) => {
	/* Define the type of issue here since IJavaScript doesn't (easily) support TypeScript
	type Issue = {
		number: number,
		body: string,
		title: string,
	}

	type files = Map<string, {
		path: string,
		content: string,
	}>
	*/
	
	let initialPrompt = `# Listed below are:
# 1. Explanation of how two files need to be changed
# 2. The original files, separated by a '---' string
# 3. An updated version of the files with the described changes, with each file separated by a '---' string
# 4. A '####' string, indicating the end of this document


## Description of issues:
${issue.body}\n`;

	// append the files to the prompt
	initialPrompt += `
## Original files:
`;
	let i = 0 ;
	for (const [fileName, file] of files) {
		initialPrompt += `# @${fileName}\n${file.content}\n`;
		// only place the delimiting string if in-between files
		if (files.size > 1 && i < files.size - 1) {
			initialPrompt += '---\n';
		}
		i++;
	}
	
	initialPrompt += `
## Updated files:
`;
	return initialPrompt;
};

v.prompt = f.buildPrompt(v.asynced.issues[0], v.filesFromIssue);


'# Listed below are:\n' +
  '# 1. Explanation of how two files need to be changed\n' +
  "# 2. The original files, separated by a '---' string\n" +
  "# 3. An updated version of the files with the described changes, with each file separated by a '---' string\n" +
  "# 4. A '####' string, indicating the end of this document\n" +
  '\n' +
  '\n' +
  '## Description of issues:\n' +
  '`@file1:grafana/base/datasource.yaml`\n' +
  '`@file2:grafana/base/grafana-route.yaml`\n' +
  '\n' +
  'The Prometheus data source needs to increase its `jsonData.timeInterval` from 5s to 10s. \n' +
  'Grafana-route needs to use `dev-grafana.operate-first.cloud` as its host, and use targetPort 3434. \n' +
  '\n' +
  '## Original files:\n' +
  '# @file1\n' +
  'apiVersion: integreatly.org/v1alpha1\n' +
  'kind: GrafanaDataSource\n' +
  'metadata:\n' +
  '  name: datasource\n' +
  'spec:\n' +
  '  name: prometheus-grafanadatasource.yaml\n' +
  '  datasources:\n' +
  '    - name: Prometheus\n' +
  '    - access

### Outputting the prompt

For later purposes, we will save the prompt from above and place it in here for reference:

```
# Listed below are:
# 1. Explanation of how two files need to be changed
# 2. The original files, separated by a '---' string
# 3. An updated version of the files with the described changes, with each file separated by a '---' string
# 4. A '####' string, indicating the end of this document


## Description of issues:
`@file1:grafana/base/datasource.yaml`
`@file2:grafana/base/grafana-route.yaml`

The Prometheus data source needs to increase its `jsonData.timeInterval` from 5s to 10s. 
Grafana-route needs to use `dev-grafana.operate-first.cloud` as its host, and use targetPort 3434. 

## Original files:
# @file1
apiVersion: integreatly.org/v1alpha1
kind: GrafanaDataSource
metadata:
  name: datasource
spec:
  name: prometheus-grafanadatasource.yaml
  datasources:
    - name: Prometheus
    - access: proxy
      editable: true
      isDefault: true
      jsonData:
        httpHeaderName1: 'Authorization'
        timeInterval: 5s
        tlsSkipVerify: true
      name: Prometheus
      secureJsonData:
        httpHeaderValue1: 'Bearer ${BEARER_TOKEN}'
      type: prometheus
      url: 'https://thanos-querier.openshift-monitoring.svc.cluster.local:9091'

---
# @file2
kind: Route
apiVersion: route.openshift.io/v1
metadata:
  name: grafana
  annotations:
    kubernetes.io/tls-acme: "true"
spec:
  host: grafana.operate-first.cloud
  to:
    kind: Service
    name: grafana-service
  port:
    targetPort: 3000


## Updated files:


```

## Generating New Files
Now that our prompt has been built, we can send it off to OpenAI's Codex API.

This will autocomplete our results and (hopefully) perform the necessary changes as described by the GitHub issue.

In [13]:
f.completionToFiles = (completions, files) => {
	// go through the list of completions, extract the filename and set the updated content
	for (const completion of completions) {``
		// extract the file tag from the completion
		const fileTagRegex = /# \@(.+)/g;
		const match = fileTagRegex.exec(completion);
		if (match !== null) {
			const fileTag = match[1];
			if (files.has(fileTag)) {
				// find the line containing the fileTag and remove all lines up to and including the fileTag 
				const lines = completion.split('\n');
				let i = 0;
				for (const line of lines) {
					i++;	
					if (line.includes(fileTag)) {
						break;
					}
				}
				// remove the lines from the completion
				const newCompletion = lines.slice(i).join('\n');
				// set the updated content
				files.get(fileTag).updatedContent = newCompletion;
			};
		}
	}
};

[Function (anonymous)]

In [14]:
var axios = require('axios');
f.getCompletion = async (prompt, maxTokens, stopSequences) => {
	stopSequences = stopSequences || v.const.OPENAI_STOP_SEQUENCES;
	const headers = {
		"Authorization": `Bearer ${v.const.OPENAI_API_KEY}`,
		"Content-Type": "application/json",
	};
	// console.log("headers", headers);
	const body = {
		prompt: prompt,
		max_tokens: maxTokens | 128,
		stop: stopSequences,
		temperature: 0.12,
		top_p: 1,
		frequency_penalty: 0,
		presence_penalty: 0,
	};
	let completionFiles;

	// request the openai api using axios
	await axios.post(v.const.OPENAI_API_URL, body, { headers }).then(async (response) => {
		// update the object with the competion result
		if (response.status == 200 && response.data.choices) {
			// awiat for response to be parsed
			if (response.data.choices.length > 0) {
				let completion = response.data.choices[0].text;
				// split completion into files 
				completionFiles = completion.split('---');
			} else {
				console.error("no completion found");
			}
		}
	});
	return completionFiles;
};

[AsyncFunction (anonymous)]

In [15]:
// get a completion from OpenAI and use it to update the files
f.updateFilesFromCompletion = async (files, prompt, maxTokens) => {
	const completion = await f.getCompletion(prompt, maxTokens);
	v.completions = completion;
	f.completionToFiles(completion, files);
}

[AsyncFunction (anonymous)]

In [16]:
v.filesFromIssue.forEach((file, name) => {
	console.log(`${name}:\n${file.updatedContent}`);
});


file1:

file2:



undefined

In [17]:
$$.async();

{
	f.updateFilesFromCompletion(v.filesFromIssue, v.prompt, 512);
}

undefined

## Conversing With The Bot

Sure we can generate content based on an issue, but what would we do if the bot wasn't happy with the results?
There needs to be a way to have back-and-forth conversation with the bot, so that we can get the bot to sculpt the PR without needing to edit the files ourselves.

We can modify the above prompt to take user-input for feedback. Since we're limited in the amount of data we can pass back-and-forth with OpenAI (4096 tokens in total),
we'll need to compress the information from the conversation.

We can try the following approaches:
1. Add a comments section in the prompt above each file so we can just place the feedback with the corresponding file.
2. Place the comments appended at the prompt prefixed with the filetag (e.g. `@file1`)


### Working with Existing Data

Not sure how things will work with the GitHub API but we'll cross that road when we get there. 

For the time being, we will resort to the following scenario:

We started out with the following prompt originally:: 

```
# Listed below are:
# 1. Explanation of how two files need to be changed
# 2. The original files, separated by a '---' string
# 3. An updated version of the files with the described changes, with each file separated by a '---' string
# 4. A '####' string, indicating the end of this document


## Description of issues:
`@file1:grafana/base/datasource.yaml`
`@file2:grafana/base/grafana-route.yaml`

The Prometheus data source needs to increase its `jsonData.timeInterval` from 5s to 10s. 
Grafana-route needs to use `dev-grafana.operate-first.cloud` as its host, and use targetPort 3434. 

## Original files:
# @file1
apiVersion: integreatly.org/v1alpha1
kind: GrafanaDataSource
metadata:
  name: datasource
spec:
  name: prometheus-grafanadatasource.yaml
  datasources:
    - name: Prometheus
    - access: proxy
      editable: true
      isDefault: true
      jsonData:
        httpHeaderName1: 'Authorization'
        timeInterval: 5s
        tlsSkipVerify: true
      name: Prometheus
      secureJsonData:
        httpHeaderValue1: 'Bearer ${BEARER_TOKEN}'
      type: prometheus
      url: 'https://thanos-querier.openshift-monitoring.svc.cluster.local:9091'

---
# @file2
kind: Route
apiVersion: route.openshift.io/v1
metadata:
  name: grafana
  annotations:
    kubernetes.io/tls-acme: "true"
spec:
  host: grafana.operate-first.cloud
  to:
    kind: Service
    name: grafana-service
  port:
    targetPort: 3000


## Updated files:

```


And we received the following files as a response:
```
# @file1
apiVersion: integreatly.org/v1alpha1
kind: GrafanaDataSource
metadata:
  name: datasource
spec:
  name: prometheus-grafanadatasource.yaml
  datasources:
    - name: Prometheus
    - access: proxy
      editable: true
      isDefault: true
      jsonData:
        httpHeaderName1: 'Authorization'
        timeInterval: 10s
        tlsSkipVerify: true
      name: Prometheus
      secureJsonData:
        httpHeaderValue1: 'Bearer ${BEARER_TOKEN}'
      type: prometheus
      url: 'https://thanos-querier.openshift-monitoring.svc.cluster.local:9091'

--- 
# @file2
kind: Route
apiVersion: route.openshift.io/v1
metadata:
  name: grafana
  annotations:
    kubernetes.io/tls-acme: "true"
spec:
  host: grafana.operate-first.cloud
  to:
    kind: Service
    name: grafana-service
  port:
    targetPort: 3434
```


### Approach #2: Appending Comments to Issue

This approach seems to be more straightforward & scalable. 
We simply encapsulate the comments in a comment section, and append it to the issue.

For the file inputs, we would use the updated versions that we received from OpenAI. 

Suppose that we saw the response from above, and recognized that the GrafanaDataSource needs an extra Prometheus datasource. We would then comment this on the issue tagging the file with the `@file1` tag:

```
`@file1` needs an extra Prometheus datasource pulling from this URL: `https://my.new.openshift.datasource.svc.cluster.local:9091`. This can use the same settings as the existing Prometheus datasource.
```

The above comment would be appended to the issue in prompt using the following format:

```
## Description of issues:
`@file1:grafana/base/datasource.yaml`
`@file2:grafana/base/grafana-route.yaml`

The Prometheus data source needs to increase its `jsonData.timeInterval` from 5s to 10s. 
Grafana-route needs to use `dev-grafana.operate-first.cloud` as its host, and use targetPort 3434. 

Revisions:
`@file1` needs an extra Prometheus datasource pulling from this URL: `https://my.new.openshift.datasource.svc.cluster.local:9091`. This can use the same settings as the existing Prometheus datasource.

```

In [18]:

v.prompt = `# Listed below are:
# 1. Explanation of how two files need to be changed
# 2. The original files, separated by a '---' string
# 3. An updated version of the files with the described changes, with each file separated by a '---' string
# 4. A '####' string, indicating the end of this document
## Description of issues:
\`@file1:grafana/base/datasource.yaml\`
\`@file2:grafana/base/grafana-route.yaml\`

\`@file1\` needs an extra Prometheus datasource pulling from this URL: \`https://my.new.openshift.datasource.svc.cluster.local:9091\`. This can use the same settings as the existing Prometheus datasource.

## Original files:
# @file1
${v.filesFromIssue.get('file1').updatedContent.trim()}
---
# @file2
${v.filesFromIssue.get('file2').updatedContent.trim()}

## Updated Files:
`;

'# Listed below are:\n' +
  '# 1. Explanation of how two files need to be changed\n' +
  "# 2. The original files, separated by a '---' string\n" +
  "# 3. An updated version of the files with the described changes, with each file separated by a '---' string\n" +
  "# 4. A '####' string, indicating the end of this document\n" +
  '## Description of issues:\n' +
  '`@file1:grafana/base/datasource.yaml`\n' +
  '`@file2:grafana/base/grafana-route.yaml`\n' +
  '\n' +
  '`@file1` needs an extra Prometheus datasource pulling from this URL: `https://my.new.openshift.datasource.svc.cluster.local:9091`. This can use the same settings as the existing Prometheus datasource.\n' +
  '\n' +
  '## Original files:\n' +
  '# @file1\n' +
  'apiVersion: integreatly.org/v1alpha1\n' +
  'kind: GrafanaDataSource\n' +
  'metadata:\n' +
  '  name: datasource\n' +
  'spec:\n' +
  '  name: prometheus-grafanadatasource.yaml\n' +
  '  datasources:\n' +
  '    - name: Prometheus\n' +
  '    - access: proxy\n' +
  

In [19]:
// fetch the new files
$$.async();

{
	f.updateFilesFromCompletion(v.filesFromIssue, v.prompt, 512);
}

undefined

#### Drumroll Please...

In [20]:
v.filesFromIssue.forEach((file, name) => {
	console.log(`${name}:\n${file.updatedContent}`);
});

file1:
apiVersion: integreatly.org/v1alpha1
kind: GrafanaDataSource
metadata:
  name: datasource
spec:
  name: prometheus-grafanadatasource.yaml
  datasources:
    - name: Prometheus
    - access: proxy
      editable: true
      isDefault: true
      jsonData:
        httpHeaderName1: 'Authorization'
        timeInterval: 10s
        tlsSkipVerify: true
      name: Prometheus
      secureJsonData:
        httpHeaderValue1: 'Bearer ${BEARER_TOKEN}'
      type: prometheus
      url: 'https://my.new.openshift.datasource.svc.cluster.local:9091'

file2:
kind: Route
apiVersion: route.openshift.io/v1
metadata:
  name: grafana
  annotations:
    kubernetes.io/tls-acme: "true"
spec:
  host: grafana.operate-first.cloud
  to:
    kind: Service
    name: grafana-service
  port:
    targetPort: 3434




undefined

In [21]:
v.filesFromIssue

Map(2) {
  'file1' => {
    path: 'grafana/base/datasource.yaml',
    content: 'apiVersion: integreatly.org/v1alpha1\n' +
      'kind: GrafanaDataSource\n' +
      'metadata:\n' +
      '  name: datasource\n' +
      'spec:\n' +
      '  name: prometheus-grafanadatasource.yaml\n' +
      '  datasources:\n' +
      '    - name: Prometheus\n' +
      '    - access: proxy\n' +
      '      editable: true\n' +
      '      isDefault: true\n' +
      '      jsonData:\n' +
      "        httpHeaderName1: 'Authorization'\n" +
      '        timeInterval: 5s\n' +
      '        tlsSkipVerify: true\n' +
      '      name: Prometheus\n' +
      '      secureJsonData:\n' +
      "        httpHeaderValue1: 'Bearer ${BEARER_TOKEN}'\n" +
      '      type: prometheus\n' +
      "      url: 'https://thanos-querier.openshift-monitoring.svc.cluster.local:9091'\n",
    updatedContent: 'apiVersion: integreatly.org/v1alpha1\n' +
      'kind: GrafanaDataSource\n' +
      'metadata:\n' +
      '  name: data